# от Бондаренко Алексея (bondaleksey@gmail.com)

Цель данного ноутбука провести моделирование распространения болезни по аэропортам и посмотреть некоторые функции библиотеки работы с графами NetworkX


Основные пункты задания: 

* реализация функции моделирование распространения болезни,
* оценка скорости распространения болезни,
* исследние графа распространения (при p = 0.5),
* заключение.

In [2]:
import numpy as np 
import pandas as pd

import matplotlib.pyplot as plt

#technical 
%matplotlib inline
import warnings
import contextlib
import time

# 1. Данные рейсов между аэропортами

In [33]:
# 
filename = "../data/airline_dec_2008_50k.csv"
df = pd.read_csv(filename, low_memory=False)
print(df.iloc[:10,:10])
print(df.iloc[:10,10:])
# print(df.iloc[20100:20110,:10])
# print(df.iloc[20100:20110,10:])

   Unnamed: 0  Year  Month  DayofMonth  DayOfWeek  DepTime  CRSDepTime  \
0           1  2008     12           1          1      NaN        1000   
1           2  2008     12           1          1      NaN        1000   
2           3  2008     12           1          1      NaN        1000   
3           4  2008     12           1          1      NaN        1000   
4           5  2008     12           1          1      NaN        1000   
5           6  2008     12           1          1      NaN        1005   
6           7  2008     12           1          1      NaN        1015   
7           8  2008     12           1          1      NaN        1020   
8           9  2008     12           1          1      NaN        1029   
9          10  2008     12           1          1      NaN        1030   

   ArrTime  CRSArrTime UniqueCarrier  
0      NaN        1100            WN  
1      NaN        1110            US  
2      NaN        1125            MQ  
3      NaN        1227       

In [6]:
print(df.shape)

(50000, 30)


In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 30 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Unnamed: 0         50000 non-null  int64  
 1   Year               50000 non-null  int64  
 2   Month              50000 non-null  int64  
 3   DayofMonth         50000 non-null  int64  
 4   DayOfWeek          50000 non-null  int64  
 5   DepTime            32968 non-null  float64
 6   CRSDepTime         50000 non-null  int64  
 7   ArrTime            31418 non-null  float64
 8   CRSArrTime         50000 non-null  int64  
 9   UniqueCarrier      50000 non-null  object 
 10  FlightNum          50000 non-null  int64  
 11  TailNum            44556 non-null  object 
 12  ActualElapsedTime  31367 non-null  float64
 13  CRSElapsedTime     50000 non-null  int64  
 14  AirTime            31367 non-null  float64
 15  ArrDelay           31367 non-null  float64
 16  DepDelay           329

In [12]:
df.describe()

,Unnamed: 0,Year,Month,DayofMonth,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,FlightNum,...,Distance,TaxiIn,TaxiOut,Cancelled,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay
count,50000.000000,50000.0,50000.0,50000.000000,50000.000000,32968.000000,50000.000000,31418.000000,50000.000000,50000.000000,...,50000.000000,31418.000000,32458.000000,50000.000000,50000.000000,4157.000000,4157.000000,4157.000000,4157.000000,4157.000000
mean,25000.500000,2008.0,12.0,11.350900,3.695000,752.043102,978.930920,791.598829,1076.869720,2592.855480,...,572.158120,7.166879,16.336866,0.355580,0.017080,12.044503,2.584316,18.813327,0.152273,7.957662
std,14433.901067,0.0,0.0,7.843122,1.889499,364.602722,496.930255,193.315743,495.563167,2116.729997,...,446.647873,5.539380,9.846548,0.478693,0.129571,31.444616,13.186963,21.499273,2.125548,29.794825
min,1.000000,2008.0,12.0,1.000000,1.000000,1.000000,5.000000,1.000000,1.000000,1.000000,...,30.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,12500.750000,2008.0,12.0,5.000000,2.000000,612.000000,630.000000,742.000000,805.000000,791.000000,...,261.000000,4.000000,10.000000,0.000000,0.000000,0.000000,0.000000,3.000000,0.000000,0.000000
50%,25000.500000,2008.0,12.0,10.000000,4.000000,657.000000,735.000000,831.000000,910.000000,2021.000000,...,446.000000,6.000000,14.000000,0.000000,0.000000,0.000000,0.000000,17.000000,0.000000,0.000000
75%,37500.250000,2008.0,12.0,17.000000,5.000000,750.000000,1300.000000,914.000000,1315.000000,4114.000000,...,733.000000,9.000000,20.000000,1.000000,0.000000,14.000000,0.000000,25.000000,0.000000,0.000000
max,50000.000000,2008.0,12.0,31.000000,7.000000,2400.000000,2359.000000,2359.000000,2359.000000,7829.000000,...,3303.000000,156.000000,206.000000,1.000000,1.000000,1092.000000,209.000000,326.000000,96.000000,336.000000


In [29]:
flights_scheduled = df[["Origin","Dest", "DayofMonth", "CRSDepTime", "CRSArrTime"]]
flights_scheduled.head()

,Origin,Dest,DayofMonth,CRSDepTime,CRSArrTime
0,HOU,DAL,1,1000,1100
1,LGA,BOS,1,1000,1110
2,SAN,SJC,1,1000,1125
3,MEM,CVG,1,1000,1227
4,ORD,DTW,1,1000,1227


In [30]:
flights_actual = df[["Origin","Dest", "DayofMonth", "DepTime", "ArrTime"]]
flights_actual.head()

,Origin,Dest,DayofMonth,DepTime,ArrTime
0,HOU,DAL,1,NaN,NaN
1,LGA,BOS,1,NaN,NaN
2,SAN,SJC,1,NaN,NaN
3,MEM,CVG,1,NaN,NaN
4,ORD,DTW,1,NaN,NaN


In [31]:
flights_actual.isna().sum()

Origin            0
Dest              0
DayofMonth        0
DepTime       17032
ArrTime       18582
dtype: int64

In [39]:
#claer actual flights 
caf = flights_actual.dropna()

In [40]:
caf.isna().sum()

Origin        0
Dest          0
DayofMonth    0
DepTime       0
ArrTime       0
dtype: int64

In [38]:
caf.shape

(31418, 5)

In [41]:
caf.describe()

,DayofMonth,DepTime,ArrTime
count,31418.000000,31418.000000,31418.000000
mean,7.202623,715.246833,791.598829
std,4.687993,314.211801,193.315743
min,1.000000,1.000000,1.000000
25%,3.000000,610.000000,742.000000
50%,7.000000,654.000000,831.000000
75%,10.000000,740.000000,914.000000
max,31.000000,2400.000000,2359.000000


In [44]:
caf.tail()

,Origin,Dest,DayofMonth,DepTime,ArrTime
49995,TUS,PHX,13,805.0,900.0
49996,DFW,IAH,13,805.0,906.0
49997,MDW,OMA,13,805.0,925.0
49998,DFW,STL,13,805.0,938.0
49999,RDU,JFK,13,806.0,935.0


In [49]:
scaf = caf.sort_values(by=['DayofMonth', 'DepTime', 'ArrTime'])

In [50]:
scaf.head()

,Origin,Dest,DayofMonth,DepTime,ArrTime
19411,MDW,BNA,1,1.0,115.0
19412,ATL,EWR,1,1.0,200.0
18829,DEN,IAD,1,2.0,509.0
18830,SLC,ATL,1,2.0,534.0
19483,ORD,MEM,1,4.0,128.0


In [51]:
scaf.tail()

,Origin,Dest,DayofMonth,DepTime,ArrTime
24709,IAD,DEN,31,2221.0,8.0
24710,IAD,MCO,31,2224.0,26.0
24711,LAX,LAS,31,2307.0,19.0
24713,DFW,AUS,31,2327.0,9.0
24712,JFK,ROC,31,2327.0,35.0


In [52]:
scaf.isna().sum()

Origin        0
Dest          0
DayofMonth    0
DepTime       0
ArrTime       0
dtype: int64

In [53]:
scaf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 31418 entries, 19411 to 24712
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Origin      31418 non-null  object 
 1   Dest        31418 non-null  object 
 2   DayofMonth  31418 non-null  int64  
 3   DepTime     31418 non-null  float64
 4   ArrTime     31418 non-null  float64
dtypes: float64(2), int64(1), object(2)
memory usage: 1.4+ MB


In [54]:
scaf.describe()

,DayofMonth,DepTime,ArrTime
count,31418.000000,31418.000000,31418.000000
mean,7.202623,715.246833,791.598829
std,4.687993,314.211801,193.315743
min,1.000000,1.000000,1.000000
25%,3.000000,610.000000,742.000000
50%,7.000000,654.000000,831.000000
75%,10.000000,740.000000,914.000000
max,31.000000,2400.000000,2359.000000
